In [23]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.pipeline import Pipeline
from sksurv.compare import compare_survival
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.metrics import cumulative_dynamic_auc, concordance_index_censored

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

In [2]:
# Lendo os dados
data = pd.read_csv('../data/processed/Processed_Telecom_Data.csv')

Levando em conta os nossos dados, podemos inferir o seguinte:
- Temos dados censurados à esquerda, pois não sabemos quando o nosso 
relacionamento com os clientes começou.
- Também temos dados censorados à direita, representados pelos clientes que
ainda não cancelaram o serviço.

Como primeiro passo, vamos aplicar o teste de Kepler-Meier para estimar a
função de sobrevivência dos nossos dados. Para tal, iremos usar apenas os 
usuários que não deram churn, já que o teste fiunciona bem em dados censurados à
direita. Como esse é um teste não paramétrico, não precisamos nos preocupar com
a distribuição dos dados. Além disso, só podemos usar duas variáveis, uma para
o tempo e outra para o evento, que no nosso caso é o cancelamento do serviço.
Para tal, usaremos a coluna `tenure` e a coluna `Churn`, respectivamente.

### Teste de Kepler-Meier

In [11]:
# Inicializando o método
kmf = KaplanMeierFitter()

# Ajustando os dados
kmf.fit(data['tenure'], data['Churn'])

# Plotando o gráfico com o plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                         y=kmf.survival_function_['KM_estimate'],
                         mode='lines',
                         name='KM estimate'))

# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=False
)

A probabilidade geral de sobrevivência é bem alta, se mantendo acima de 50% 
durante todo o período. Nos primeiros 4 meses, a probabilidade do cliente não 
vivenciar o evento de interesse (Churn) é acima de 90%.

Agora, vamos ver se há diferença entre os grupos de gênero e senioridade.

In [25]:
# Inicializando o método
kmf = KaplanMeierFitter()

# Initializando a figure
fig = go.Figure()

# Ajustando os dados
for genero in data['gender'].unique():
    data_gender = data.query(f"gender == '{genero}'")
    kmf.fit(data_gender['tenure'], data_gender['Churn'], label=genero)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                                y=kmf.survival_function_[f'{genero}'],
                                mode='lines',
                                name=genero))
    
# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para os diferentes gêneros</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)

In [33]:
# Dividindo os dados por gênero 
data_masculino = data.query("gender == 'Male'")
data_feminino = data.query("gender == 'Female'")

# Aplicando o teste
resultados = logrank_test(data_masculino['tenure'], data_feminino['tenure'], data_masculino['Churn'], data_feminino['Churn'])

if resultados.p_value < 0.05:
    print(f"Com um p valor de {resultados.p_value}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {resultados.p_value}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 0.46841735075696966, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


No geral, pessoas do gênero masculino têm uma probabilidade de churn maior do
que pessoas do gênero feminino. No entanto, a diferença não é tão grande. Também 
é possível notar que o cenário se inverte com clientes com um tempo de 
relacionamento superior a 67 meses, com as pessoas do gênero feminino tendo uma
probabilidade de churn maior do que as pessoas do gênero masculino.

Com 95% de confiança, podemos dizer que as curvas de churn para os grupos de
gênero são iguais.

In [60]:
# Inicializando o método
kmf = KaplanMeierFitter()

# Initializando a figure
fig = go.Figure()

# Ajustando os dados
for senioridade in data['SeniorCitizen'].unique():
    data_senior = data.query(f"SeniorCitizen == {senioridade}")
    if senioridade == 1:
        label = 'Senior'
        kmf.fit(data_senior['tenure'], data_senior['Churn'], label=label)
        fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                                y=kmf.survival_function_[f'{label}'],
                                mode='lines',
                                name=label))
    else:
        label = 'Não senior'
        kmf.fit(data_senior['tenure'], data_senior['Churn'], label=label)
        fig.add_trace(go.Scatter(x=kmf.survival_function_.index,
                                y=kmf.survival_function_[f'{label}'],
                                mode='lines',
                                name=label))
    
# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para os diferentes grupos de idade</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)

In [62]:
# Filtrando os dados de clientes sênior
data_senior = data.query("SeniorCitizen == 1")

# Aplicando o teste
tempo_senior, probabilidade_senior = kaplan_meier_estimator(data_senior['Churn'], data_senior['tenure'])

# Filtrando os dados de clientes não sênior
data_not_senior = data.query("SeniorCitizen == 0")

# Aplicando o teste
tempo_not_senior, probabilidade_not_senior = kaplan_meier_estimator(data_not_senior['Churn'], data_not_senior['tenure'])

# Plotando o gráfico
fig = go.Figure(data=[go.Scatter(x=tempo_senior, y=probabilidade_senior, name='Senior'),
                      go.Scatter(x=tempo_not_senior, y=probabilidade_not_senior, name='Não Senior')])

# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para os diferentes grupos de idade</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)  

In [63]:
# Aplicando o teste
resultados = logrank_test(data_senior['tenure'], data_not_senior['tenure'], data_senior['Churn'], data_not_senior['Churn'])

if resultados.p_value < 0.05:
    print(f"Com um p valor de {resultados.p_value}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {resultados.p_value}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 1.2676192066669992e-25, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


Pessoas mais velhas são mais propensas a dar churn do que pessoas mais novas.
A diferença durante o último mês do período chegou a ser de 20%. 

Com 95% de confiança, podemos dizer que as curvas de churn para os grupos de 
senioridade são significativamente diferentes.

Agora, vamos considerar as variáveis 'Partner' e 'Dependents', para averiguar se
há diferença entre pessoas que moram sozinhas e pessoas que moram com alguém.

In [9]:
# Filtrando os dados de pessoas com parceiros
data_partner = data.query("Partner == 'Yes'")

# Aplicando o teste
tempo_partner, probabilidade_partner = kaplan_meier_estimator(data_partner['Churn'], data_partner['tenure'])

# Filtrando os dados de pessoas sem parceiros
data_not_partner = data.query("Partner == 'No'")

# Aplicando o teste
tempo_not_partner, probabilidade_not_partner = kaplan_meier_estimator(data_not_partner['Churn'], data_not_partner['tenure'])

# Plotando o gráfico
fig = go.Figure(data=[go.Scatter(x=tempo_partner, y=probabilidade_partner, name='Sim'),
                      go.Scatter(x=tempo_not_partner, y=probabilidade_not_partner, name='Não')])


# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para pessoas com parceiros</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)
    

In [10]:
# Comparando os grupos
partner = data['Partner']

# Aplicando o teste
chi2, p_valor = compare_survival(array_status, partner)

if p_valor < 0.05:
    print(f"Com um p valor de {p_valor}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {p_valor}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 4.13295113442747e-94, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


A probabilidade de sobrevivência para pessoas que possuem parceiros é maior do
que para pessoas que não possuem parceiros. A diferença é de aproximadamente
20% durante boa parte do período.

Com 95% de confiança, podemos dizer que as curvas de churn para os grupos de
parceiros são significativamente diferentes.

In [11]:
# Filtrando os dados de pessoas com dependentes
data_dependents = data.query("Dependents == 'Yes'")

# Aplicando o teste
tempo_dependents, probabilidade_dependents = kaplan_meier_estimator(data_dependents['Churn'], data_dependents['tenure'])

# Filtrando os dados de pessoas sem dependentes
data_not_dependents = data.query("Dependents == 'No'")

# Aplicando o teste
tempo_not_dependents, probabilidade_not_dependents = kaplan_meier_estimator(data_not_dependents['Churn'], data_not_dependents['tenure'])

# Plotando o gráfico
fig = go.Figure(data=[go.Scatter(x=tempo_dependents, y=probabilidade_dependents, name='Sim'),
                      go.Scatter(x=tempo_not_dependents, y=probabilidade_not_dependents, name='Não')])


# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para pessoas com dependentes</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)
    

In [12]:
# Comparando os grupos
dependent = data['Dependents']

# Aplicando o teste
chi2, p_valor = compare_survival(array_status, dependent)

if p_valor < 0.05:
    print(f"Com um p valor de {p_valor}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {p_valor}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 1.5372382701501665e-52, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


A probabilidade de sobvrevivência de pessoas com dependentes é maior
do que para pessoas sem dependentes. A diferença durante o último mês do período
chegou a pouco menos de 25%.

Com 95% de confiança, podemos dizer que as curvas de churn para os grupos de
dependentes são significativamente diferentes.

Por fim, vamos analisar como a probabilidade de sobrevivência varia de acordo
com o tipo de contrato.

In [13]:
# Filtrando os dados de pessoas com dependentes
data_contract_month = data.query("Contract == 'Month-to-month'")

# Aplicando o teste
tempo_contract_month, probabilidade_contract_month = kaplan_meier_estimator(data_contract_month['Churn'], data_contract_month['tenure'])

# Filtrando os dados de pessoas sem dependentes
data_contract_two_year = data.query("Contract == 'Two year'")

# Aplicando o teste
tempo_contract_two_year, probabilidade_contract_two_year = kaplan_meier_estimator(data_contract_two_year['Churn'], data_contract_two_year['tenure'])

# Filtrando os dados de pessoas sem dependentes
data_contract_one_year = data.query("Contract == 'One year'")

# Aplicando o teste
tempo_contract_one_year, probabilidade_contract_one_year = kaplan_meier_estimator(data_contract_one_year['Churn'], data_contract_one_year['tenure'])

# Plotando o gráfico
fig = go.Figure(data=[go.Scatter(x=tempo_contract_month, y=probabilidade_contract_month, name='Month-to-month'),
                      go.Scatter(x=tempo_contract_two_year, y=probabilidade_contract_two_year, name='Two years'),
                      go.Scatter(x=tempo_contract_one_year, y=probabilidade_contract_one_year, name='One year')])


# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Análise de probabilidade dos coeficiente de Keplen-Meier para diferentes tipos de contrato</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=True
)
    

In [14]:
# Comparando os grupos
dependent = data['Contract']

# Aplicando o teste
chi2, p_valor = compare_survival(array_status, dependent)

if p_valor < 0.05:
    print(f"Com um p valor de {p_valor}, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")
    
else:
    print(f"Com um p valor de {p_valor}, não podemos rejeitar a hipótese nula de que os grupos são significativamente iguais")

Com um p valor de 0.0, podemos rejeitar a hipótese nula de que os grupos são significativamente iguais


Contratos mensais têm uma probabilidade de sobrevivência menor do que contratos
anuais. A probabilidade de sobrevivência ficou abaixo de 50% no trigésimo quinto
mês para contratos mensais.

Com 95% de confiança, podemos dizer que pelo menos uma das curvas de churn para 
os grupos de contrato é significativamente diferente.

### Treinando um modelo multivariado
Até aqui, usamos apenas uma variável para estimar a probabilidade de 
sobrevivência. Agora, vamos usar uma abordagem multivariada, usando todas as
variáveis disponíveis.

In [32]:
# Dividiando os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(data.drop('Churn', axis=1),
                                                       array_status,
                                                        test_size=0.2,
                                                        random_state=200,
                                                        stratify=data['Churn'])

# Selecionando as variáveis categóricas
cat_cols = x_treino.select_dtypes(include='object').columns

# Criando um pipeline para as variáveis categóricas
cat_pipe = Pipeline([('onehot', OneHotEncoder(drop='first'))])

# Criando um transformer para as variáveis categóricas
transformer = ColumnTransformer([('cat', cat_pipe, cat_cols)])


In [33]:
# Transformando os dados de treino
x_treino = transformer.fit_transform(x_treino)

# Transformando os dados de teste
x_teste = transformer.transform(x_teste)

In [37]:
# Inicializando o modelo com regularization
modelo = CoxPHSurvivalAnalysis(alpha=0.01)

# Treinando o modelo
modelo.fit(x_treino, y_treino)

# Prevendo a probabilidade de sobrevivência
pred_sobrevivencia = modelo.predict_survival_function(x_treino)

# Prevendo as estimativas de risco
pred_risco = modelo.predict(x_treino)


In [ ]:
# Plotando o gráfico de sobrevivência para cada cliente
fig = go.Figure()
for num in range(len(pred_sobrevivencia)):
    fig.add_trace(go.Scatter(x=pred_sobrevivencia[num].x, y=pred_sobrevivencia[num].y))

# Atualizando o layout da figure
fig.update_layout(
    title='<b>Probabilidade de Sobrevivência dado o tempo de relacionamento<br>com a empresa</b><br><sup><i>Predição do tempo de sobrevida de cada cliente</i></sup>',
    title_font=dict(size=20, family="Arial"),
    font_color='#646369',
    template='plotly_white',
    yaxis=dict(showticklabels=True, showgrid=True, title='Probabilidade de Sobrevivência'),
    xaxis=dict(showgrid=False, title='Tempo em meses'),
    title_x=0.1,
    width=910,
    height=500,
    showlegend=False
)

fig

In [73]:
# Avaliando o index c
c_index = concordance_index_censored(y_treino['status'], y_treino['time'], pred_risco)

# Avaliando o AUC dinâmico
tempo = np.arange(1, 72, 1)

# Calculando o AUC dinâmico
auc_forest, auc_medio_forest = cumulative_dynamic_auc(y_treino, y_treino, pred_risco, tempo)

ValueError: censoring survival function is zero at one or more time points

In [56]:
# Inicializando o modelo
modelo_forest = RandomSurvivalForest(random_state=200)

# Treinando o modelo
modelo_forest.fit(x_treino, y_treino)

# Fazendo previsões com o modelo
pred_risco_forest = modelo_forest.predict(x_treino)


RandomSurvivalForest(random_state=200)

In [61]:
# Avaliando o index c
c_index = concordance_index_censored(y_treino['status'], y_treino['time'], pred_risco_forest)

# Calculando o AUC dinâmico
auc_forest, auc_medio_forest = cumulative_dynamic_auc(y_treino, y_treino, pred_risco_forest, tempo)

ValueError: censoring survival function is zero at one or more time points